In [ ]:
import numpy as np
import cv2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score

# Function to extract frames from a video
def extract_frames(video_path, max_frames=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))
        frame = frame.astype('float32') / 255.0  # Normalize
        frames.append(frame)
    cap.release()
    return np.array(frames)

# Function to prepare the dataset
def prepare_dataset(video_paths, labels, max_frames=30):
    X, y = [], []
    for video, label in zip(video_paths, labels):
        frames = extract_frames(video, max_frames)
        if len(frames) == max_frames:
            X.append(frames)
            y.append(label)
    X = np.array(X)
    y = np.array(y)

    lb = LabelBinarizer()
    y = lb.fit_transform(y)

    return train_test_split(X, y, test_size=0.2, random_state=42)

# Function to create the model
def create_model():
    model = Sequential()

    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in vgg16.layers:
        layer.trainable = False  # Freeze VGG16 layers

    model.add(TimeDistributed(vgg16, input_shape=(30, 224, 224, 3)))
    model.add(TimeDistributed(Flatten()))

    model.add(LSTM(128, return_sequences=False))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Preparing the dataset
video_paths = ["video1.mp4", "video2.mp4"]  # Example video file paths
labels = ["real", "fake"]  # Example labels corresponding to the videos

X_train, X_test, y_train, y_test = prepare_dataset(video_paths, labels)

# Creating and training the model
model = create_model()
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=8)

# Saving the trained model in the recommended format
model.save('/content/deepfake_detection_model.keras')
model.save('/content/deepfake_detection_model.h5')
# Evaluating the model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


In [ ]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model

# Function to extract frames from the video (as defined earlier)
def extract_frames(video_path, max_frames=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))
        frame = frame.astype('float32') / 255.0  # Normalize
        frames.append(frame)
    cap.release()
    return np.array(frames)

# Load your trained model
model = load_model('/content/deepfake_detection_model.keras')  # Load the .keras model

# Path to the video you want to test
video_path = "/content/video1.mp4"

# Extract frames from the video
frames = extract_frames(video_path)

# Ensure the video has the required number of frames
if len(frames) == 30:
    frames = np.expand_dims(frames, axis=0)  # Add batch dimension
    print("Input shape for prediction:", frames.shape)  # Check the input shape
    prediction = model.predict(frames)

    if prediction > 0.5:
        print("The video is a deepfake.")
    else:
        print("The video is real.")
else:
    print("The video does not have enough frames for analysis. Please use a longer video.")
